In [2]:
## IMPORT LIBRARIES

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

C:\Users\iliac\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
## IMPORT DATA

In [5]:
data = pd.read_excel("SaqmeBot.xlsx")

In [6]:
## SET VARIABLES

In [7]:
columns = ['District','Area','Rooms','Floor','Number_of_floors','Condition','Status','Project','Heat','Gas','Storage','Cellar','Balcony','Garage','gancxadeba','Price']

In [8]:
train_columns = ['District','Area','Rooms','Floor','Number_of_floors','Condition','Status','Project','Heat','Gas','Storage','Cellar','Balcony','Garage','gancxadeba']

In [9]:
amenities = ['Heat','Gas','Storage','Cellar','Balcony','Garage']

In [10]:
cat =  ['District','Condition','Status','Project'] 

In [11]:
## TRANSLATE COLUMN NAMES

In [12]:
data.columns = columns

In [13]:
## PreProcessing

In [14]:
# Change Nan to 0 in Amenities

In [15]:
data[amenities] = data[amenities].fillna(0)

In [16]:
# Drop other Nan-s

In [17]:
data.dropna(inplace=True)

In [18]:
# Drop 'gancxadebebi'. We don't need that feature

In [19]:
data.drop('gancxadeba',axis=1,inplace=True)

In [20]:
# Remove Fake data

In [21]:
data = data[(data['Area']>10) & (data['Area']<300)]  # Keep 'Area' from 10 to 300
data = data[(data['Floor']>1) & (data['Floor']<30)] # Keep 'Floor' from 1 to 30
data = data[(data['Number_of_floors']>1) & (data['Number_of_floors']<30)] # Keep 'Number_of_floors' from 1 to 30
data = data[(data['Price']>5000) & (data['Price']<2000000)] # Keep 'Price' from 5,000 to 2,000,000

In [22]:
# Create Dummies for Categorical Variables and drop first column for every feature

In [23]:
data = pd.get_dummies(data,columns=cat, drop_first=True)

In [24]:
# Split Label

In [25]:
labels = data['Price'] # Create Label
data.drop('Price',axis=1,inplace=True) # Delete 'Price' column from Data

In [26]:
# Scale Features

In [27]:
scaler = StandardScaler()
#data = scaler.fit_transform(data)
data[data.columns] = scaler.fit_transform(data[data.columns])

In [28]:
## Split train and test

In [29]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [35]:
## Neural Network

In [36]:
import tensorflow as tf

In [37]:
sess = tf.InteractiveSession()

In [38]:
data.shape

(46953, 83)

In [39]:
inputs = tf.placeholder(tf.float32,shape=[None,83])
output = tf.placeholder(tf.float32,shape=[None,])

In [40]:
hidden_size = 256

input_weights = tf.Variable(tf.truncated_normal([83,hidden_size]))
input_biases = tf.Variable(tf.zeros(hidden_size))

output_weights = tf.Variable(tf.truncated_normal([hidden_size,1]))
output_biases = tf.Variable(tf.zeros([1,1]))

In [41]:
hidden_layer = tf.nn.relu(tf.matmul(inputs,input_weights)+input_biases)
output_layer = tf.matmul(hidden_layer,output_weights)+output_biases

In [42]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=output))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [43]:
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_function)

In [44]:
correct_prediction = tf.equal(tf.argmax(output_layer,1), tf.argmax(output,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [47]:
tf.global_variables_initializer().run()

In [48]:
training_epochs = 5000
display_step = 1000
batch_size = 32

In [52]:
for epoch in range(training_epochs):
    avg_cost = 0.0
    total_batch = int(len(X_train) / batch_size)
    x_batches = np.array_split(X_train, total_batch)
    y_batches = np.array_split(y_train, total_batch)
    for i in range(total_batch):
        batch_x, batch_y = x_batches[i], y_batches[i]
        _, c = sess.run([optimizer, loss_function],feed_dict={inputs: batch_x, output: batch_y})
        avg_cost += c / total_batch
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", \
            "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Accuracy:", accuracy.eval({x: x_test, y: y_test}))

InvalidArgumentError: Input to reshape is a tensor with 33 values, but the requested shape has 1
	 [[Node: gradients/softmax_cross_entropy_with_logits_sg/Reshape_2_grad/Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/Mean_grad/truediv, gradients/softmax_cross_entropy_with_logits_sg/Reshape_2_grad/Shape)]]

Caused by op 'gradients/softmax_cross_entropy_with_logits_sg/Reshape_2_grad/Reshape', defined at:
  File "C:\Users\iliac\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\iliac\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\iliac\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\iliac\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\iliac\Anaconda3\lib\site-packages\ipykernel\kernela